# Time Series - Maker workshop

## Quick round table

Presentation & expectations?

## Definition

- **Time series data is data that is collected at different points in time.** This is opposed to cross-sectional data which observes individuals, companies, etc. at a single point in time.


- If you previously followed the *Maker workshop dedicated to Machine Learning*, you've already worked with cross-sectional data, but not time series.


- Time series can be found in a wide variety of domains: in economics, social sciences, medicine, but also ( and obviously) in physical sciences and engineering. As a result, **we deal with them a lot at Total!**

## Outline

1. Today's challenge
2. Today's Data Science environment checklist
3. Exploring the data 
    - Types, indexes and unique values
    - Distributions
    - Correlations
4. Dealing with missing values
5. Resampling techniques
6. Time series visualization
7. Anomalies detection techniques
8. Forecasting
8. Open discussion / work session

## Today's Challenge

**Predict the air temperature in 2017 based on weather data from 2009 to 2016.**

- Features available:
    - Air temperature
    - Atmospheric pressure
    - Humidity
    - Wind direction
    - Etc.

## Today's Data Science environment checklist

- A Jupyter notebook
- The data folder (the one that we sent)
- The following libraries installed:

In [ ]:
! make -f ../setup/Makefile

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from fbprophet import Prophet

# Optional
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Uncomment this if you don't have the data
# !wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
# !unzip jena_climate_2009_2016.csv.zip

# Used for data preparation

#raw_data_bis = pd.read_csv('../data/jena_climate_2009_2016.csv')
#raw_data_bis['open_st'] = 1.0
#
#df = raw_data_bis[['VPmax (mbar)', 'VPact (mbar)']].copy()
#import random
#ix = [(row, col) for row in range(df.shape[0]) for col in range(df.shape[1])]
#for row, col in random.sample(ix, int(round(.01*len(ix)))):
#    df.iat[row, col] = np.nan
#
#raw_data_bis[['VPmax (mbar)', 'VPact (mbar)']] = df.copy()
#
#raw_data_bis.to_csv('../data/jena_climate_2009_2016.csv', index=False)

## Exploring the data

### Reading the raw data

- `head -n 10` is a useful shell command to give a look at a file's header (the first 10 lines in this case)
- In a Jupyter notebook, we can use the symbol `!` to run shell commands

_What are the useful details that you can see thanks to this command?_

In [ ]:
!head -n 10 ../data/jena_climate_2009_2016.csv

Now that we have a better idea of the file's format, we can implement our reading function:

In [ ]:
raw_data = pd.read_csv('../data/jena_climate_2009_2016.csv', sep=',')
raw_data.head()

### Data types

- Checking for data types is useful to make sure that types were properly inferred when reading the raw CSV file
- If you've already explored the data, you can specify the undetected types in the `pandas.read_csv` function
- Tip: Casting to smaller float types can help you tremendly reduce the size of a dataset

_Comment on the following dtypes. Do you think the proper types were inferred?_

In [ ]:
raw_data.dtypes

### Indexing

- When dealing with time series, we'll see that it can be useful to make the most out of pandas' `DatetimeIndex`, i.e. to set a `Datetime` column as index of the dataframe.

_Let's verify if the Datetime type was correctly inferred from the CSV file._

In [ ]:
type(raw_data['Date Time'][0])

### Checking duplicated rows

- Before to continue the data manipulation, we should check for potential duplicated rows in the data that we want to get rid of.

_What is the percentage of duplicated rows among the complete dataset?_

In [ ]:
percentage = raw_data[raw_data.duplicated(subset=['Date Time'])].shape[0] / raw_data.shape[0] * 100
print(f'Among the complete data, {round(percentage * 100, 2)}% are duplicated rows.')

In [ ]:
raw_data.drop_duplicates(subset='Date Time', inplace=True)

### Unique values

- Checking for unique values will give you information on your variables' granularity:
    - A small number of unique values can indicate the presence of a category
    - A single unique value may indicate that a variable is never changing, even out of your sample
   
_Do you notice any of these two cases in your dataset?_

In [ ]:
for col in raw_data.columns: 
    print(col, ' '*(20-len(col))+'----->', len(raw_data[col].unique()))

### Distributions

- With the `seaborn` library, we can easily plot the distributions and the relationship between each pair of sensors

_Let's give a look at the following graph: from your functional knowledge of the sensors, can you identify normal or abnormal patterns?_

In [ ]:
SELECTED_COLUMNS = ['p (mbar)', 'T (degC)', 'H2OC (mmol/mol)', 'sh (g/kg)', 'wd (deg)']

In [ ]:
sns.pairplot(raw_data[SELECTED_COLUMNS])

plt.show()

### Correlations

- Correlation analysis is a statistical method used to **evaluate the strength of relationship between two quantitative variables**. 


- A high correlation means that two or more variables have a strong relationship with each other.
- A weak correlation means that the variables are hardly related.

_Let's continue our analysis by plotting the correlation matrix. Do you notice anything?_

In [ ]:
def print_correlation(df):
    corr = df.corr()
    
    plt.figure(figsize=(8, 8))
    
    ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0,
                     cmap=sns.diverging_palette(20, 220, n=200),
                     square=True, annot=True)
    
    ax.set_xticklabels(ax.get_xticklabels(),
                       rotation=45,
                       horizontalalignment='right')
    
    plt.show()

In [ ]:
print_correlation(raw_data[SELECTED_COLUMNS])

Here, we use our own custom function to get a stylized correlation matrix. However, you could simply use the pandas method _your_dataframe_name.corr()_.

## Dealing with missing values

### Identification of missing values

_Let's identify missing values in our dataset._

Tip: Knowing that our variables are floats, the missing values will appear as `NaN` (= Not A Number).

In [ ]:
raw_data.isna().sum()

In [ ]:
raw_data[raw_data.isna().any(axis=1)]

## Resampling techniques

### Assess the time series delta

In [ ]:
sorted_data = raw_data.copy()
sorted_data['Date Time'] = pd.to_datetime(sorted_data['Date Time'], format='%d.%m.%Y %H:%M:%S')
sorted_data =  sorted_data.sort_values('Date Time')

dt_index_delta = sorted_data['Date Time'].diff()
dt_index_delta.value_counts()

Lucky for us, we have is a nice `resample()` method for pandas dataframes that have a DatetimeIndex.

### Create a DatetimeIndex

In [ ]:
reindexed_data = sorted_data.copy()
reindexed_data.set_index('Date Time', inplace=True)

In order to better illustrate the concept of resampling, let's create a fake sinusoidal time series.

In [ ]:
def create_fake_ts(length=10000):
    raw_values = np.sin(np.linspace(1, length, length)*2*np.pi/(24*60))
    date = pd.date_range(start='2017-01-01', periods=len(raw_values), freq='min')
    
    sampled = [i for i in range(len(raw_values)//2, len(raw_values), 8*60)]
    sampled_date = date[sampled]

    raw_less_values = raw_values[sampled]

    raw_df = pd.DataFrame(raw_values[:sampled[0]])
    raw_df.index = date[:sampled[0]]

    raw_less_values_df = pd.DataFrame(raw_less_values)
    raw_less_values_df.index = sampled_date

    raw_df = raw_df.append(raw_less_values_df)
    
    raw_df.index.name = 'date'
    raw_df.columns = ['values']

    return raw_df

In [ ]:
ts_to_resample = create_fake_ts(length=10000)

We can then apply the same analysis of the time differences as in the previous section.

In [ ]:
ts_to_resample.reset_index(inplace=True)

ts_to_resample['delta'] = ts_to_resample['date'].diff()

ts_to_resample.set_index('date', inplace=True)

ts_to_resample.head()

In [ ]:
plt.figure(figsize=(15, 5))

plt.scatter(x=ts_to_resample.index, y=ts_to_resample['values'])

plt.title('Time series to resample')

plt.show()

In [ ]:
plt.figure(figsize=(15, 5))

(ts_to_resample['delta'].dt.total_seconds() / (3600 * 24)).plot()

plt.title('Differences between timestamps')

plt.show()

In this example data, starting on 2017-01-04, we no longer receive 1 data point every minute but instead 1 data point every 8h.

In our case, we have 2 different ways to go to obtain a time series with equally-spaced data points:
1. Remove some data points from the first part of the time series in order to get 8h-spaced data points similar to the second part of the time series: this technique is known as **undersampling**.
2. Add some data points in the second part of the time series in order to get 1min-spaced data points similar to the first part of the time series: this technique is known as **oversampling**.

### Oversampling

#### Forward-fill method

Even though it depends on the problem at hand, a way to go can be to use the frequency the most represented in order not to add/remove too many data points.

When oversampling your time series, i.e. creating new data points, you'll need to make a decision regarding which values to assign to these new points.

When resampling time series, a common risk is to introduce **data leakage** by adding data from the future to the past, i.e. data that would not have been available at the time.

For example, if you decide to fill a missing data point at time t with the next available values (this is known as **backward filling**), how would you have been able to do that at time t, knowing that these future values were not available at the time?

You should always ask yourself this question when manipulating time series, especially when adding data points and creating new features.

![](../setup/images/ffill.png)

As presented on this diagram, you should always use **forward filling** as a resampling method when oversampling. Backward filling would bring in unavailable values from the future and introduce a data leak.

In [ ]:
ts_to_resample_min = ts_to_resample['values'].resample('T').ffill()

plt.figure(figsize=(15, 5))

plt.plot(ts_to_resample_min, color='orange')
plt.scatter(x=ts_to_resample.index, y=ts_to_resample['values'])

plt.title('Resampling with the forward fill method')

plt.show()

#### Other methods - Linear

In [ ]:
ts_to_resample_min = ts_to_resample['values'].resample('T').interpolate(method='linear')

plt.figure(figsize=(15, 5))

plt.plot(ts_to_resample_min, color='orange')
plt.scatter(x=ts_to_resample.index, y=ts_to_resample['values'])

plt.title('Resampling with the linear method')

plt.show()

#### Other methods - Nearest

In [ ]:
ts_to_resample_min = ts_to_resample['values'].resample('T').interpolate(method='nearest')

plt.figure(figsize=(15, 5))

plt.plot(ts_to_resample_min, color='orange')
plt.scatter(x=ts_to_resample.index, y=ts_to_resample['values'])

plt.title('Resampling with the "nearest" method')

plt.show()

As explained earlier, if you look closely at the different graphs, you'll realize that **the forward fill method is the only method presented which doesn't introduce any data leak.**

### Mix of undersampling and oversampling

In [ ]:
ts_to_resample_min = ts_to_resample['values'].resample('4H').ffill()

plt.figure(figsize=(15, 5))

plt.plot(ts_to_resample_min, color='orange')
plt.scatter(x=ts_to_resample.index, y=ts_to_resample['values'])

plt.show()

Now, we know everything we need to resample our data on a 10-min basis.

In [ ]:
clean_data = reindexed_data.resample('10min').ffill()

## Time series visualization

### Global view & analysis

In [ ]:
plt.figure(figsize=(20, 7))

for col in clean_data.columns:
    plt.plot(clean_data[col], label=col)

plt.legend(loc='upper right')
plt.title('Sensor values')

plt.show()

### Sensor-level view & analysis

In [ ]:
for col in clean_data.columns:
    
    plt.figure(figsize=(20, 7))
    plt.plot(clean_data[col])
    plt.title(col)
    
    plt.show()

## Anomalies detection

### Definition

- An anomaly is an outlier data point, which does not follow the collective common pattern of the majority of the data points and hence can be easily separated or distinguished from the rest of the data.

- In our case, we can try to identify abnormal temperatures over the period.

In [ ]:
clean_data.head()

### Fix threshold

In [ ]:
TAG_NAME = 'T (degC)'

plt.figure(figsize=(20, 7))

plt.plot(clean_data[TAG_NAME])
plt.title(TAG_NAME)

plt.show()

_What could be a relevant threshold to apply to this specific sensor ?_

Now, consider that we apply the following thresholds (upper/lower) for the specified sensors.

In [ ]:
SENSORS_THRESHOLDS = {TAG_NAME:[-15, 34]}

Let's backtest our **fix threshold** strategy:

In [ ]:
backtesting_df = clean_data.copy()

for col in SENSORS_THRESHOLDS.keys():
    upper_alert = (backtesting_df[col] > SENSORS_THRESHOLDS[col][1])
    lower_alert = (backtesting_df[col] < SENSORS_THRESHOLDS[col][0])
    
    backtesting_df[f'is_alert_{col}'] = (upper_alert | lower_alert).astype(int)

In [ ]:
plt.figure(figsize=(20, 7))

plt.plot(backtesting_df[f'is_alert_{TAG_NAME}'], color='red')
plt.title(f'Fix threshold - Alerting state on sensor {TAG_NAME}')

plt.show()

### Statistical profiling

- Creating a statistical profile of the data can be the fastest and the most useful approach, and it still offers a **clear and explainable outcome**.

- In the case of statistical profiling, **we use the mean, median, standard deviations and/or quantiles to come up with upper and lower bounds** to detect anomalies.

In [ ]:
plt.figure(figsize=(20, 5))

sns.boxplot(clean_data[TAG_NAME])

plt.show()

Now, consider that we use the 1st and 99th quantiles for the specified sensors.

In [ ]:
QUANTILE_PARAM = 0.99

upper_quantile = clean_data[TAG_NAME].quantile(QUANTILE_PARAM)
lower_quantile = clean_data[TAG_NAME].quantile(1-QUANTILE_PARAM)

Let's backtest our **statistical profiling** strategy:

In [ ]:
backtesting_df = clean_data.copy()

for col in SENSORS_THRESHOLDS.keys():
    upper_alert = (backtesting_df[col] > upper_quantile)
    lower_alert = (backtesting_df[col] < lower_quantile)
    
    backtesting_df[f'is_alert_{col}'] = (upper_alert | lower_alert).astype(int)

In [ ]:
plt.figure(figsize=(20, 7))

plt.plot(backtesting_df[f'is_alert_{TAG_NAME}'], color='red')
plt.title(f'Statistical profiling - Alerting state on sensor {TAG_NAME}')

plt.show()

## Forecasting

### A word of caution

One needs to be careful when predicting the future:

- _"Stocks have reached what looks like a permanently high plateau."_ - Irving Fischer, Professor of Economics, Yale University, 1929
    - True or False?

- _"Computers in the future weigh no more than 1.5 tons."_ - Popular Mechanics, forecasting the relentless march of science, 1949
    - True or False?

### Introduction to Prophet

- Open-sourced by Facebook's core data science team a few years ago, Prophet is based on time series decomposition but has the ability to model different seasonalities as well as the effect of holidays and special events.

- On [Prophet Github page](https://github.com/facebook/prophet), we find the following description:

_"Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well."_

- In this section, we'll try to assess how Prophet performs to predict the future value of the temperature (the T (degC) sensor).

The input to Prophet is always a DataFrame with 2 columns: `ds` and `y`:
- The `ds` (datestamp) column should be of a format expected by pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp. 
- The `y` column must be numeric, and represents the measurement we wish to forecast.

In [ ]:
TAG_NAME = 'T (degC)'

prophet_df = clean_data.resample('1d').ffill()
prophet_df = prophet_df[[TAG_NAME]].reset_index()
prophet_df = prophet_df.rename(columns={'Date Time':'ds', TAG_NAME:'y'})

prophet_df.head()

Prophet follows the sklearn model API. We create an instance of the `Prophet `class and then call its `fit` and `predict` methods.

In [ ]:
model = Prophet()
model.fit(prophet_df)

Now that we have a model, we can make predictions on a DataFrame with a column `ds` containing the dates for which a prediction is to be made. 

You can get a suitable DataFrame that extends into the future a specified number of days using the helper method `Prophet.make_future_dataframe` (by default, it will also include the dates from the history).

In [ ]:
future = model.make_future_dataframe(periods=365)
future.tail()

Now, we can apply the `predict` method to this DataFrame: it will assign each row a predicted value which it names `yhat`. If you pass in historical dates, it will provide an in-sample fit.

In [ ]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

We can lot the forecast by calling the `Prophet.plot` method and passing in our forecast DataFrame.

In [ ]:
fig1 = model.plot(forecast)

If you want to see the forecast components, you can use the `Prophet.plot_components` method. 

By default you’ll see the trend, yearly seasonality, and weekly seasonality of the time series. If you include holidays, you’ll see those here, too.

In [ ]:
fig2 = model.plot_components(forecast)

## Thank you!
### Any feedback? Return on time invested?